In [1]:
import torch
from pyPDMP.models import VAE
from pyPDMP.utils import loss_function

import matplotlib.pyplot as plt
import torchvision
import torchvision.datasets as datasets

In [2]:
m = VAE()

In [3]:
m

VAE(
  (fc1): Linear(in_features=2, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=2, bias=True)
)

## Training the VAE

In [12]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.0, 0.0, 0.0), (1.0, 1.0, 1.0))
    ])

dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

In [14]:
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=32, 
                                          shuffle=True)

In [18]:
size = 28*28

def train(m, train_loader, epoch):
    optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)
    m.train()
    train_loss = 0
    for batch_idx, (x, _) in enumerate(train_loader):
        # flatten the batch
        x = x.to(device).view(-1, size)
        optimizer.zero_grad()
        x.reshape()
        recon_batch, mu, logvar = m(x)
        loss = loss_function(recon_batch, x, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(x)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [19]:
train(m, data_loader, 10)

RuntimeError: output with shape [1, 28, 28] doesn't match the broadcast shape [3, 28, 28]